# Printing/drawing Theano graphs

Following:

http://deeplearning.net/software/theano/tutorial/printing_drawing.html

The functions `theano.printing.pprint()` and `theano.printing.debugprint()` can print a graph to the terminal before or after compilation. We also have `pydotprint()` to create images of the function.

See also

http://deeplearning.net/software/theano/library/printing.html#libdoc-printing

Consider again a simple logistic regression example:

In [1]:
import numpy
import theano
import theano.tensor as T

In [2]:
rng = numpy.random

# training data
N = 400
feats = 784
D = (rng.randn(N, feats).astype(theano.config.floatX), rng.randint(size=N, low=0, high=2).astype(theano.config.floatX))
training_steps = 10000

# symbolic vars
x = T.matrix('x')
y = T.vector('y')
w = theano.shared(rng.randn(feats).astype(theano.config.floatX), name='w')
b = theano.shared(numpy.asarray(0., dtype=theano.config.floatX), name='b')
x.tag.test_value = D[0]
y.tag.test_value = D[1]

# construct the expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w) - b))     # probability of having a 1
prediction = p_1 > 0.5                      # predict 0 or 1

# compute gradients
xent = -y * T.log(p_1) - (1 - y) * T.log(1 - p_1)   # cross entropy loss function
cost = xent.mean() + 0.01 * (w**2).sum()            # cost to optimize
gw, gb = T.grad(cost, [w, b])

# training and prediction functions
train = theano.function(inputs=[x, y], outputs=[prediction, xent], 
                       updates=[[w, w - 0.01 * gw], [b, b - 0.01 * gb]], name='train')
predict = theano.function(inputs=[x], outputs=prediction, name='predict')

## Pretty printing

In [3]:
theano.printing.pprint(prediction)

'gt((TensorConstant{1} / (TensorConstant{1} + exp(((-(x \\dot w)) - b)))), TensorConstant{0.5})'

## Debug printing

In [4]:
theano.printing.debugprint(prediction)

Elemwise{gt,no_inplace} [@A] ''   
 |Elemwise{true_div,no_inplace} [@B] ''   
 | |DimShuffle{x} [@C] ''   
 | | |TensorConstant{1} [@D]
 | |Elemwise{add,no_inplace} [@E] ''   
 |   |DimShuffle{x} [@F] ''   
 |   | |TensorConstant{1} [@D]
 |   |Elemwise{exp,no_inplace} [@G] ''   
 |     |Elemwise{sub,no_inplace} [@H] ''   
 |       |Elemwise{neg,no_inplace} [@I] ''   
 |       | |dot [@J] ''   
 |       |   |x [@K]
 |       |   |w [@L]
 |       |DimShuffle{x} [@M] ''   
 |         |b [@N]
 |DimShuffle{x} [@O] ''   
   |TensorConstant{0.5} [@P]


The post-compilation graph:

In [5]:
theano.printing.debugprint(predict)

Elemwise{Composite{GT(scalar_sigmoid((-((-i0) - i1))), i2)}} [@A] ''   4
 |CGemv{inplace} [@B] ''   3
 | |Alloc [@C] ''   2
 | | |TensorConstant{0.0} [@D]
 | | |Shape_i{0} [@E] ''   1
 | |   |x [@F]
 | |TensorConstant{1.0} [@G]
 | |x [@F]
 | |w [@H]
 | |TensorConstant{0.0} [@D]
 |InplaceDimShuffle{x} [@I] ''   0
 | |b [@J]
 |TensorConstant{(1,) of 0.5} [@K]


## Picture printing of graphs

Pre-compilation:

In [6]:
theano.printing.pydotprint(prediction, outfile='logreg_pydotprint_prediction', var_with_name_simple=True)

The output file is available at logreg_pydotprint_prediction.png


<img src='logreg_pydotprint_prediction.png'>

Post-compilation:

In [8]:
theano.printing.pydotprint(predict, outfile='logreg_pydotprint_predict', var_with_name_simple=True)

The output file is available at logreg_pydotprint_predict.png


<img src='logreg_pydotprint_predict.png'>

The optimized training graph:

In [9]:
theano.printing.pydotprint(train, outfile='logreg_pydotprint_train', var_with_name_simple=True)

The output file is available at logreg_pydotprint_train.png


<img src="logreg_pydotprint_train.png">